# Housing Unit Allocation with Person Record File Full Workflow

## Overview
This code works with the National Structures Inventory to run the housing unit allocation (HUA) and the person record file (PREC) workflow.
The HUA process is generalizable to any county in the United States. The HUA process will work for any file that has locations of structures and some basic information about the buildings.
The process is designed to work with [IN-CORE](https://incore.ncsa.illinois.edu/), a community resilience modeling environment.
Using IN-CORE requires an account and access to the IN-CORE Dataservice.

Functions are provided to obtain and clean data required for the version 2 Housing Unit Allocation. 

## Required Inputs
Program requires the following inputs:
If using the National Structures Inventory there are no required inputs.
    
## Output Description
The output of this workflow is a CSV file with the housing unit inventory allocated to a building inventory using the housing unit allocation model.

The output CSV is designed to be used in the Interdependent Networked Community Resilience Modeling Environment (IN-CORE).

IN-CORE is an open source python package that can be used to model the resilience of a community. To download IN-CORE, see:

https://incore.ncsa.illinois.edu/


## Instructions
Users can run the workflow by executing each block of code in the notebook.

## Description of Program
- program:    ncoda_07fv1_HUA_PREC_NSI
- task:       Start with NSI building inventory, run housing unit allocation algorithm, and then run person record file algorithm
- See github commits for description of program updates
- Current Version: v1 - 
- 2024-02-20 - Combine code from 07c, 07d, and 07e into one notebook
- 2024-05-22 - removed the drop down menu, did not work consistently
- project:    Interdependent Networked Community Resilience Modeling Environment (IN-CORE), Subtask 5.2 - Social Institutions
- funding:	  NIST Financial Assistance Award Numbers: 70NANB15H044 and 70NANB20H008 
- author:     Nathanael Rosenheim

## Required Citations:
Rosenheim, Nathanael, Roberto Guidotti, Paolo Gardoni & Walter Gillis Peacock. (2021). Integration of detailed household and housing unit characteristic data with critical infrastructure for post-hazard resilience modeling. _Sustainable and Resilient Infrastructure_. 6(6), 385-401. https://doi.org/10.1080/23789689.2019.1681821

Rosenheim, Nathanael (2021) “Detailed Household and Housing Unit Characteristics: Data and Replication Code.” _DesignSafe-CI_. 
https://doi.org/10.17603/ds2-jwf6-s535.

In [1]:
# To reload submodules need to use this magic command to set autoreload on
%load_ext autoreload
%autoreload 2
from pyncoda.ncoda_00g_community_options import *
from IPython.display import display

### How to set up the Community Dictionary
Please review the python code in the file pyncoda/ncoda_00g_community_options.py

In this file you will find a collection of data dictionaries with various ways to setup the inputs for the Housing Unit Allocation process. 

The basic dictionary includes the name of the community, the county FIPS code, your input building inventory file, and key variables in the building inventory file.

In [2]:
# select a community from this list
# if your community is not in this list, add it to the file ncoda_00g_community_options.py
list_community_options(communities_dictionary)

['Lumberton, NC: IN-CORE Building inventory for Robeson County, NC',
 'Galveston, TX: IN-CORE Building inventory for Galveston County, TX',
 'Galveston, TX: NSI Building inventory for Galveston County, TX',
 'Galveston, TX: IN-CORE Building inventory for Galveston Island, TX',
 'Mayfield, KY: NSI Building inventory for Graves County, KY',
 'Beaumont, TX: NSI Building inventory for Jefferson County, TX',
 'Beaumont, TX: Safayet Building inventory for Jefferson County, TX',
 'Pentwater, MI: NSI Building inventory for Oceana County, MI',
 'Seaside, OR: NSI Building inventory for Clatsop County, OR',
 'Lane County, OR: NSI Building inventory for Lane County, OR',
 'Benton County, OR: NSI Building inventory for Benton County, OR',
 'Southeast Texas Urban Integrated Field Lab: NSI Building inventory for Southeast Texas',
 'Brazos County, TX: NSI Building inventory for Brazos County, TX']

In [3]:
community_id_by_name = 'Seaside, OR: NSI Building inventory for Clatsop County, OR'

In [4]:
community_id, focalplace, countyname, countyfips = get_community_id_by_name(community_id_by_name)
communities = {community_id : communities_dictionary[community_id]}

Selected community ID: Seaside_OR_NSI
Seaside, OR is in OREGON
Focal place: Seaside
Seaside, OR is in Clatsop County, OR with FIPS code 41007
Use IN-CORE: False


## Setup Python Environment

In [5]:
import pandas as pd
import geopandas as gpd # For reading in shapefiles
import numpy as np
import sys # For displaying package versions
import os # For managing directories and file paths if drive is mounted
import scooby # Reports Python environment

import contextily as cx # For adding basemap tiles to plot
import matplotlib.pyplot as plt # For plotting and making graphs

In [6]:
# open, read, and execute python program with reusable commands
from pyncoda.ncoda_00d_cleanvarsutils import *
from pyncoda.ncoda_04c_poptableresults import *
from pyncoda.ncoda_07i_process_communities import process_community_workflow

In [7]:
# Generate report of Python environment
base_packages = ['pandas','ipyleaflet','seaborn','contextily']
incore_packages = ['pyincore','pyincore_viz']
check_packages = base_packages + incore_packages
print(scooby.Report(additional=check_packages))


--------------------------------------------------------------------------------
  Date: Thu Feb 20 13:00:38 2025 Central Standard Time

                OS : Windows (10 10.0.22631 SP0 Multiprocessor Free)
            CPU(s) : 16
           Machine : AMD64
      Architecture : 64bit
               RAM : 31.7 GiB
       Environment : Jupyter

  Python 3.10.14 | packaged by Anaconda, Inc. | (main, May  6 2024, 19:44:50)
  [MSC v.1916 64 bit (AMD64)]

            pandas : 2.2.2
        ipyleaflet : Module not found
           seaborn : 0.13.2
        contextily : 1.6.0
          pyincore : Module not found
      pyincore_viz : Module not found
             numpy : 1.26.4
             scipy : 1.13.1
           IPython : 8.25.0
        matplotlib : 3.8.4
            scooby : 0.10.0

  Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303
  for Intel(R) 64 architecture applications
--------------------------------------------------------------------------------


In [8]:
# Check working directory - good practice for relative path access
os.getcwd()

'c:\\Users\\nathanael99\\MyProjects\\GitHub\\intersect-community-data'

## Run Base Housing Unit Inventory for 2020

In [9]:
# open, read, and execute python program with reusable commands
from pyncoda.ncoda_00b_directory_design import directory_design
from pyncoda.CommunitySourceData.api_census_gov.acg_01a_BaseInventory import BaseInventory

seed = 9876
version = '2.0.0'
version_text = 'v2-0-0'
basevintage = 2010
outputfolder ="OutputData"
outputfolders = {}
savefiles = True
use_incore = False

In [10]:
mutually_exclusive_varstems_roots_dictionary_lists = {}
new_char_dictionaries = {}
new_char_dictionaries['family'] = {}
new_char_dictionaries['Hispanic'] = {}

In [11]:
from pyncoda.CommunitySourceData.api_census_gov.acg_00b_hui_block2010 import *
from pyncoda.CommunitySourceData.api_census_gov.acg_00c_hispan_block2010 import *

mutually_exclusive_varstems_roots_dictionary_lists[2010] = [tenure_size_H16_varstem_roots,
                                                    vacancy_status_H5_varstem_roots,
                                                    group_quarters_P42_varstem_roots]

new_char_dictionaries['family'][2010] = [family_byrace_P18_varstem_roots]
new_char_dictionaries['Hispanic'][2010] = [tenure_size_H16HAI_varstem_roots,
                                        hispan_byrace_H7_varstem_roots,
                                        tenure_byhispan_H15_varstem_roots
                                        ]

In [14]:
# updated for 2020
from pyncoda.CommunitySourceData.api_census_gov.acg_00b_hui_block2020 import *
from pyncoda.CommunitySourceData.api_census_gov.acg_00c_hispan_block2020 import *

mutually_exclusive_varstems_roots_dictionary_lists[2020] = [tenure_size_H12_2020_varstem_roots,
                                                    vacancy_status_H5_2020_varstem_roots,
                                                    group_quarters_P18_2020_varstem_roots]

new_char_dictionaries['family'][2020] = [family_byrace_P16_2020_varstem_roots]
new_char_dictionaries['Hispanic'][2020] = [tenure_size_H12HAI_2020_varstem_roots,
                                        hispan_byrace_H7_2020_varstem_roots,
                                        tenure_byhispan_H11_2020_varstem_roots
                                        ]


In [15]:
vintage = 2020
for community in communities.keys():

    # Create empty container to store outputs for in-core
    # Will use these to combine multiple counties
    hui_incore_county_df = {}
    title = "Housing Unit Inventory v2.0.0 data for "+communities[community]['community_name']
    print("Generating",title)
    output_filename = f'hui_{version_text}_{community}_{basevintage}_rs{seed}'

    county_list = ''

    # set output folder
    outputfolders = directory_design(
                        state_county_name = community,
                        outputfolder = outputfolder)

    # community dictionary
    community_dict = communities[community]
    use_incore = community_dict['building_inventory']['use_incore']


    # Workflow for generating HUI data for IN-CORE
    for county in communities[community]['counties'].keys():
        state_county = communities[community]['counties'][county]['FIPS Code']
        state_county_name  = communities[community]['counties'][county]['Name']
        print(state_county_name,': county FIPS Code',state_county)
        county_list = county_list + state_county_name+': county FIPS Code '+state_county

        print("Generating HUI data for",state_county_name)

        # create output folders for hui data generation
        outputfolders = directory_design(state_county_name = community,
                                            outputfolder = outputfolder)
                          
        # Generate base housing unit inventory
        block_df = {}
        block_df['core'] = BaseInventory.get_apidata(state_county = state_county, 
                                                    geo_level = 'Block',
                                                    vintage = str(vintage),
                                                    mutually_exclusive_varstems_roots_dictionaries =
                                                                        mutually_exclusive_varstems_roots_dictionary_lists[vintage],
                                                    outputfolders = outputfolders,
                                                    outputfile = "CoreHUI")
        
        block_df['family'] = BaseInventory.graft_on_new_char(base_inventory= block_df['core'],
                                                            state_county = state_county,
                                                            new_char = 'family',
                                                            new_char_dictionaries = new_char_dictionaries['family'][vintage],
                                                            basevintage = str(vintage), 
                                                            basegeolevel = 'Block',
                                                            outputfile = "hui",
                                                            outputfolders = outputfolders)

       
        block_df['hispan'] = BaseInventory.graft_on_new_char(base_inventory= block_df['family'],
                                                        state_county = state_county,
                                                        new_char = 'hispan',
                                                        new_char_dictionaries = new_char_dictionaries['Hispanic'][vintage],
                                                        basevintage = str(vintage), 
                                                        basegeolevel = 'Block',
                                                        outputfile = "hui",
                                                        outputfolders = outputfolders)


Generating Housing Unit Inventory v2.0.0 data for Seaside, OR
Clatsop County, OR : county FIPS Code 41007
Generating HUI data for Clatsop County, OR
File OutputData/Seaside_OR_NSI/03_BaseInventory/CoreHUI_41007_2020.csv Already exists - Skipping API Call.
File OutputData/Seaside_OR_NSI/03_BaseInventory/hui_family_41007_2020.csv Already exists with new variable grafted - Skipping API Call.
Base Housing Unit Inventory has new characteristic hispan
Graft process will predict missing values of  hispan

***************************************
    Base Inventory has 2079 observations not set
***************************************


***************************************
    Predicting hispan based on ['numprec', 'ownershp', 'hispanbyH12HAI', 'byracehispan'] ['numprec', 'ownershp'] H12HAI
***************************************


***************************************
    Base Inventory has 2079 observations not set
***************************************

File OutputData/Seaside_OR_NSI/02

c:\Users\nathanael99\MyProjects\GitHub\intersect-community-data\pyncoda\CommunitySourceData\api_census_gov\acg_01a_BaseInventory.py:418: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41'
 '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41'
 '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41'
 '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41'
 '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41'
 '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41'
 '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41'
 '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41'
 '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41'
 '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41' '41'
 '41' '41' '41' '41' '41' '41' '41'


**********************************
Reshape dataframe to convert unit of analysis
   For mutually exclusive dataframe for: TENURE BY HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER
       Unit of analysis converted from 2173 block s, to 352 household
hispanbyH11
Fill missing values with 0
['Block2020str', 'numprec', 'ownershp']
Updating hispanbyH12HAI_counter  based on total probability and ['Block2020str', 'numprec', 'ownershp']
['Block2020str', 'race']
Updating hispanbyH7_counter  based on total probability and ['Block2020str', 'race']
['Block2020str', 'ownershp']
Updating hispanbyH11_counter  based on total probability and ['Block2020str', 'ownershp']
Merge vars for hispanbyH12HAI_counter = ['Block2020str', 'numprec', 'ownershp']
Updating flags based on hucount_hispanbyH12HAIupdated
Merge vars for hispanbyH7_counter = ['Block2020str', 'race']
Updating flags based on hucount_hispanbyH7updated
Merge vars for hispanbyH11_counter = ['Block2020str', 'ownershp']
Updating flags based on hucount_h

In [16]:
block_df['core'].head()

,huid,Block2020,Block2020str,numprec,ownershp,family,race,hispan,vacancy,gqtype,ageP18,sex,hu_counter
0,B410079501001007H001,410079501001007,B410079501001007,1.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,1
1,B410079501001007H002,410079501001007,B410079501001007,3.0,1.0,-999.0,1.0,0.0,NaN,NaN,NaN,NaN,2
2,B410079501001007H003,410079501001007,B410079501001007,4.0,1.0,-999.0,1.0,0.0,NaN,NaN,NaN,NaN,3
3,B410079501001007H004,410079501001007,B410079501001007,4.0,1.0,-999.0,1.0,0.0,NaN,NaN,NaN,NaN,4
4,B410079501001012H001,410079501001012,B410079501001012,1.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,1


In [17]:
block_df['family'].head()

,huid,Block2020,Block2020str,numprec,ownershp,family,race,hispan,vacancy,gqtype,...,hu_counter,family_flag,family_flagset,totalprob_family,familybyP16,hucount_familybyP16,sumby_familybyP16,prob_familybyP16,familybyP16_counter,hucount_familybyP16updated
0,B410079501001007H001,410079501001007,B410079501001007,1.0,1.0,0.0,1.0,0.0,NaN,NaN,...,1.0,family set to 0 by core hui,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,B410079501001012H001,410079501001012,B410079501001012,1.0,1.0,0.0,1.0,0.0,NaN,NaN,...,1.0,family set to 0 by core hui,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,B410079501001012H002,410079501001012,B410079501001012,1.0,1.0,0.0,1.0,0.0,NaN,NaN,...,2.0,family set to 0 by core hui,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,B410079501001012H003,410079501001012,B410079501001012,1.0,1.0,0.0,1.0,0.0,NaN,NaN,...,3.0,family set to 0 by core hui,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,B410079501001012H004,410079501001012,B410079501001012,1.0,1.0,0.0,1.0,0.0,NaN,NaN,...,4.0,family set to 0 by core hui,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
block_df['hispan'].head()

,huid,Block2020,Block2020str,numprec,ownershp,family,race,hispan,vacancy,gqtype,...,hispanbyH11,hucount_hispanbyH11,sumby_hispanbyH11,prob_hispanbyH11,hispanbyH12HAI_counter,hispanbyH7_counter,hispanbyH11_counter,hucount_hispanbyH12HAIupdated,hucount_hispanbyH7updated,hucount_hispanbyH11updated
0,B410079501001007H001,410079501001007,B410079501001007,1.0,1.0,0.0,1.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B410079501001012H001,410079501001012,B410079501001012,1.0,1.0,0.0,1.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B410079501001012H002,410079501001012,B410079501001012,1.0,1.0,0.0,1.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B410079501001012H003,410079501001012,B410079501001012,1.0,1.0,0.0,1.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,B410079501001012H004,410079501001012,B410079501001012,1.0,1.0,0.0,1.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
from pyncoda.ncoda_04c_poptableresults import *
# add race ethnicity to data frame for better map legends
hui_race_df = PopResultsTable.add_race_ethnicity_to_pop_df(block_df['hispan'])

In [22]:
hui_race_df.head()

,huid,Block2020,Block2020str,numprec,ownershp,family,race,hispan,vacancy,gqtype,...,hucount_hispanbyH11,sumby_hispanbyH11,prob_hispanbyH11,hispanbyH12HAI_counter,hispanbyH7_counter,hispanbyH11_counter,hucount_hispanbyH12HAIupdated,hucount_hispanbyH7updated,hucount_hispanbyH11updated,Race Ethnicity
0,B410079501001007H001,410079501001007,B410079501001007,1.0,1.0,0.0,1.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1 White alone, Not Hispanic"
1,B410079501001012H001,410079501001012,B410079501001012,1.0,1.0,0.0,1.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1 White alone, Not Hispanic"
2,B410079501001012H002,410079501001012,B410079501001012,1.0,1.0,0.0,1.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1 White alone, Not Hispanic"
3,B410079501001012H003,410079501001012,B410079501001012,1.0,1.0,0.0,1.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1 White alone, Not Hispanic"
4,B410079501001012H004,410079501001012,B410079501001012,1.0,1.0,0.0,1.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1 White alone, Not Hispanic"


In [23]:
where = communities[community_id]['community_name']
print(where, focalplace, countyname, countyfips)

Seaside, OR Seaside Clatsop County, OR 41007


In [27]:
PopResultsTable.pop_results_table(
                  input_df = hui_race_df, 
                  who = "Total Population by Households", 
                  what = "by Race, Ethnicity",
                  where = countyname,
                  when = "2020",
                  row_index = "Race Ethnicity",
                  col_index = 'Tenure Status')

Tenure Status,1 Owner Occupied (%),2 Renter Occupied (%),Total Population by Households (%)
Race Ethnicity,,,
"1 White alone, Not Hispanic","22,446 (87.5%)","11,421 (76.6%)","33,867 (83.5%)"
"2 Black alone, Not Hispanic",76 (0.3%),135 (0.9%),211 (0.5%)
"3 American Indian and Alaska Native alone, Not Hispanic",177 (0.7%),133 (0.9%),310 (0.8%)
"4 Asian alone, Not Hispanic",357 (1.4%),165 (1.1%),522 (1.3%)
"5 Other Race, Not Hispanic","1,356 (5.3%)",965 (6.5%),"2,321 (5.7%)"
"6 Any Race, Hispanic","1,252 (4.9%)","2,085 (14.0%)","3,337 (8.2%)"
Total,"25,664 (100.0%)","14,904 (100.0%)","40,568 (100.0%)"


In [36]:
hui_race_df['All'] = 'All'

PopResultsTable.pop_results_table(
                  input_df = hui_race_df, 
                  who = "Total Population by Households", 
                  what = "by Group Quarters Type",
                  where = countyname,
                  when = "2020",
                  row_index = 'Group Quarters Type',
                  col_index = 'All')

All,All (%),Total Population by Households (%)
Group Quarters Type,,
0. NA (non-group quarters),"37,231 (91.8%)","37,231 (91.8%)"
1 Correctional facilities for adults,"3,337 (8.2%)","3,337 (8.2%)"
Total,"40,568 (100.0%)","40,568 (100.0%)"


In [46]:
# list if HUID = B410079502001025H099
block_df['core'][block_df['core']['Block2020'] == '410079502001025']

,huid,Block2020,Block2020str,numprec,ownershp,family,race,hispan,vacancy,gqtype,ageP18,sex,hu_counter,Race Ethnicity
1726,B410079502001025H001,410079502001025,B410079502001025,2.0,1.0,-999.0,1.0,0.0,NaN,0.0,NaN,NaN,1,"1 White alone, Not Hispanic"
1727,B410079502001025H002,410079502001025,B410079502001025,2.0,1.0,-999.0,1.0,0.0,NaN,0.0,NaN,NaN,2,"1 White alone, Not Hispanic"
1728,B410079502001025H003,410079502001025,B410079502001025,4.0,1.0,-999.0,1.0,0.0,NaN,0.0,NaN,NaN,3,"1 White alone, Not Hispanic"
1729,B410079502001025H004,410079502001025,B410079502001025,6.0,1.0,-999.0,1.0,0.0,NaN,0.0,NaN,NaN,4,"1 White alone, Not Hispanic"
1730,B410079502001025H005,410079502001025,B410079502001025,1.0,2.0,0.0,1.0,0.0,NaN,0.0,NaN,NaN,5,"1 White alone, Not Hispanic"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15229,B410079502001025H098,410079502001025,B410079502001025,2.0,2.0,-999.0,1.0,1.0,NaN,1.0,NaN,NaN,98,"6 Any Race, Hispanic"
23017,B410079502001025H099,410079502001025,B410079502001025,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,99,NaN
23018,B410079502001025H100,410079502001025,B410079502001025,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,100,NaN
23019,B410079502001025H101,410079502001025,B410079502001025,7.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,101,NaN


In [38]:
# describe the variable gqtype
hui_race_df['gqtype'].value_counts()

gqtype
0.0    16502
1.0     1031
Name: count, dtype: int64

In [40]:

# describe the variable vacancy
hui_race_df['vacancy'].value_counts()

vacancy
5.0    4128
7.0     499
1.0     451
3.0     222
2.0     105
4.0      79
Name: count, dtype: int64

In [42]:
hui_race_df = PopResultsTable.add_vacancy_to_pop_df(hui_race_df)

PopResultsTable.pop_results_table(
                  input_df = hui_race_df, 
                  who = "Total Households", 
                  what = "by Vacancy Type",
                  where = countyname,
                  when = "2020",
                  row_index = 'Vacancy Type',
                  col_index = 'All')

All,All (%),Total Households (%)
Vacancy Type,,
1 For Rent,451 (8.2%),451 (8.2%)
"2 Rented, not occupied",105 (1.9%),105 (1.9%)
3 For sale only,222 (4.0%),222 (4.0%)
"4 Sold, not occupied",79 (1.4%),79 (1.4%)
"5 For seasonal, recreational, or occasional use","4,128 (75.3%)","4,128 (75.3%)"
7 Other vacant,499 (9.1%),499 (9.1%)
Total,"5,484 (100.0%)","5,484 (100.0%)"


In [ ]:
print("Total Population by Race and Ethnicity:")
print(f"https://data.census.gov/cedsci/table?g=050XX00US{countyfips}&tid=DECENNIALDHC2020.P5")

print("Total Vacancy Status:")
print(f"https://data.census.gov/cedsci/table?g=050XX00US{countyfips}&tid=DECENNIALDHC2020.H5")

Total Population by Race and Ethnicity:
https://data.census.gov/cedsci/table?g=050XX00US41007&tid=DECENNIALDHC2020.P5
